In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import numpy as np
import os


import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir

#  SVM module
from svm import svm as PredictSVMChoice

# 
data_dir = '/media/cat/4TBSSD/yuki/'
#data_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/'

labels = ['left_paw', 
            'right_paw',
            'nose',
            'jaw',
            'right_ear',
            'tongue',
            'lever',
            'all',
            'code_04',
            'code_04_lockout']

Autosaving every 180 seconds


In [8]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
names = ['AQ2']

# 
svm = PredictSVMChoice.PredictSVMChoice()
svm.root_dir = data_dir
svm.random_flag = False  # shuffle data to show baseline
svm.verbose = True

# window parameters
svm.window = 15              # prediction window, backwards in time in seconds
svm.lockout_window = 10      # time in sec for locking out other pulls - used to generate lockout examples

#
svm.imaging_rate = 30

#
svm.sliding_window = svm.imaging_rate      # number of frames in sliding window for SVM steps; usually 1 sec = img_rate
svm.lockout = False

# movement related params
svm.no_movement = 3          # number of seconds of quiescence before behavior initiation; parameter comes from
                             #     the "generate_movements_quiescent_periods" notebook in behavior_analysis
svm.random_lockout = 3       # number of seconds of buffer for selecting random trials for control

# pca params
svm.pca_flag = True
svm.pca_var = 0.95      # this is total amount of var explained to be used - NOT USED FOR NOW
svm.all_comps = 30      # this is the total # components saved
svm.nComp = 30          # this is the total # of comps to be used for decoding
            
# svm parameters
svm.min_trials = 10     # minimum number of trials to run SVM
svm.xvalidation = 10    # KFold xvalidation step
svm.data_split = 0.8    # split 80/20 data for training  NOT USED WITH XVALIDATION WHICH SPLITS 90/10
svm.method = 'sigmoid'  # method used for computing SVM

# run-time parameters
svm.parallel = True
svm.n_cores = svm.xvalidation
svm.overwrite = False

#

## session info
# session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
# svm.session_id = 'all'

# 
# codes = ['left_paw','right_paw','jaw']
svm.all_labels = ['left_paw', 
                  'right_paw',
                  'nose',
                  'jaw',
                  'right_ear',
                  'tongue',
                  'lever',
                  'all',
                  'code_04',
                  'code_04_lockout'
                 ]

# labels = ['code_04']  # DO NOT USE TO PREDICT LEVER - THE PCA COMPRESSION IS POOR
labels = ['left_paw']
svm.labels = labels
svm.code = '_whole_stack'
svm.session_id = 'Mar15'

svm.add_lever_to_ca_shift = False

# 
for name in names:
    svm.animal_id = name

    #
    svm.predict_whole_stack()   # this requires PCA to be run on the entire imaging session first: 400000, 128, 128
                                        # need also movement initiations 
                                        # need also shift between initiations and ca imaging
                                # This bypasses the need to compute [Ca] stms for each behavior/trial



  running:  AQ2pm_Mar15_Week2_30Hz
   [Ca] file missing, skipping ...
DONE predicting SVM on animal:  AQ2


In [ ]:
###############################################
########## VISUALIZE RESULTS ##################
###############################################

# to visualize results load the "visualize_svm-body_movements" notebook from /visualize/



In [13]:
#########################################################
########## VISUALIZE RESULTS - QUICK AND DIRTY ##########
#########################################################

import pickle

fname = '/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2pm_Mar15_Week2_30Hzcode_04_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow30.npz'
#fname = '/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2pm_Mar15_Week2_30Hz_whole_stack_pca30Components_window15sec_Xvalid10_Slidewindow30Frames_accuracy.npz'
data = np.load(fname, allow_pickle=True)



if 'AI3' in fname:
    acc = data['accuracy']
    acc_c04 = np.array(acc[8])   
    imaging_rate = 15
    clr='blue'

elif 'AQ2' in fname:
    acc_c04 = data['accuracy']
    if 'pca30' in fname:
        acc_c04 = np.array(acc_c04[8]) 
        
    imaging_rate = 30
    clr='black'

#for k in range(len(acc)):
#    print (k, labels[k], acc[k], np.array(acc[k]).shape)


print ("acc 04: ", acc_c04.shape)

######################
fig=plt.figure()
ax=plt.subplot(111)
mean_r = acc_c04.mean(1)
std_r = np.std(acc_c04,axis=1)

t = np.arange(mean_r.shape[0])/imaging_rate-(15-1)

print ("plotting")
plt.plot(t,mean_r, linewidth=3, c=clr)
ax.fill_between(t, mean_r+std_r,
               mean_r-std_r,
               color=clr,
               alpha=.2)

plt.plot([-15,+15],[0.5,0.5],'--',c='grey')
plt.plot([0,0],[0,1],'--',c='grey')

plt.ylim(0.3,1.0)
plt.xlim(-15,1)
plt.show()


acc 04:  (871, 10)
plotting


In [64]:

fnames = np.loadtxt('/media/cat/4TBSSD/yuki/AQ2/blue_light.txt',dtype='str')
s = []
for fname in fnames:
    data = np.load(fname)

    start = data['start_blue']
    end = data['end_blue']
    img = data['img_frame_triggers']
    
    s.append(start/30)

y = np.histogram(s,bins=np.arange(0,5,0.1))

plt.plot(y[1][1:],y[0])    
plt.show()
    
    

In [ ]:
# ######################################################
# ##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
# ######################################################

# # select animal names
# names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
# #names = ['IA1']

# # 
# svm = PredictSVMChoice()
# svm.root_dir = data_dir
# svm.random_flag = False  # shuffle data to show baseline
# svm.verbose = True

# # window parameters
# svm.window = 15              # prediction window, backwards in time in seconds
# svm.lockout_window = 10      # time for locking out other pulls; OPTIONAL
# svm.sliding_window = 30      # number of frames in sliding window for SVM steps; usually 1 sec = 30 frames
# svm.lockout = False

# # movement related params
# svm.no_movement = 3          # number of seconds of quiescence before behavior initiation; parameter comes from
#                              #     the "generate_movements_quiescent_periods" notebook in behavior_analysis
# svm.random_lockout = 3       # number of seconds of buffer for selecting random trials for control

# # pca params
# svm.pca_flag = True
# svm.pca_var = 0.95      # this is total amount of var explained to be used - NOT USED FOR NOW
# svm.all_comps = 30      # this is the total # components saved
# svm.nComp = 30          # this is the total # of comps to be used for decoding
            
# # svm parameters
# svm.min_trials = 10     # minimum number of trials to run SVM
# svm.xvalidation = 10    # KFold xvalidation step
# svm.data_split = 0.8    # split 80/20 data for training  NOT USED WITH XVALIDATION WHICH SPLITS 90/10
# svm.method = 'sigmoid'  # method used for computing SVM

# # run-time parameters
# svm.parallel = True
# svm.n_cores = svm.xvalidation
# svm.overwrite = False

# ## session info
# # session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
# # svm.session_id = 'all'
# svm.session_id = 'all'

# # 
# # codes = ['left_paw','right_paw','jaw']
# labels = ['left_paw', 
#           'right_paw',
#           'nose',
#           'jaw',
#           'right_ear',
#           'tongue',
#           'lever',
#           'all',
#           'code_04',
#           'code_04_lockout']

# svm.labels = labels
# svm.code = '_whole_stack'

# # 
# for name in names:
#     svm.animal_id = name

#     #
#     svm.predict_whole_stack()   # this requires PCA to be run on the entire imaging session first: 400000, 128, 128
#                                         # need also movement initiations 
#                                         # need also shift between initiations and ca imaging
#                                 # This bypasses the need to compute [Ca] stms for each behavior/trial


In [28]:
data = np.load('/home/cat/concatenated_res.npz',allow_pickle=True)
cf = data['conf_matrix_ten']
print (cf.shape)

y_pred = data['y_predicted']
print (y_pred.shape)
print (y_pred[2])

(10, 30, 30)
(10,)
[ 7.  2. 12. 21. 22. 23. 21.  5.  7.  4. 15.  1. 22.  8.  1. 29. 23. 15.
 16. 10. 29. 20. 15.  6. 24. 28. 20.  2. 28. 20.  9. 21. 17. 14. 17. 27.
  8. 24. 28. 12. 15. 29. 14.  6. 13. 24.  2. 19. 12. 23. 12. 12. 19. 24.
  9. 15. 22. 15. 10. 23.  6.  4. 29. 14. 15. 10.  1. 23. 18.  4. 26.  6.
 18.  0. 20.  9. 23. 12. 10. 29. 24. 24. 22. 20. 18. 15.  5. 27.  7. 23.
  7. 15.  1.  9.  7.  5.  4. 10.  8.  4. 18. 20. 18. 13.  6. 14. 27. 17.
  6. 12. 28. 21. 11. 25. 16. 21. 15.  7.  0. 20.  1.  0. 21. 26.  7. 28.
  7.  7. 29. 14.  2.  6. 27.  1. 16. 20.  6.  2. 19.  8. 21. 23.  8. 19.
 25. 21. 27. 13. 10. 16. 23. 22. 29.  1.  5. 21.  4.  7.  1. 24.  2.  9.
  9. 13.  5. 20. 24.  8. 15. 12. 27. 13. 21. 16. 24. 20. 20. 12. 24.  8.
 27.  6. 29. 29.  7. 16. 18. 10. 27.  4.  0. 18. 24.  5. 22. 12. 27. 12.
 21.  2. 17.  1.  7. 10. 24. 20. 16.  6.  0. 25. 19. 23. 28.  5.  4.  0.
 15.  4. 28. 24. 21. 15.  9.  5. 28. 29. 10.  2. 20. 14.  8. 14. 12.  4.
  4. 13.  6.  2. 16. 27. 24.  4.

In [29]:
ten_svm = data['ten_svm']
print (ten_svm.shape)
print (ten_svm)

KeyError: 'ten_svm is not a file in the archive'

In [30]:
 def plot_decision_time(ax=None):

        #
        if ax is None:
            ax=plt.subplot(111)
        clrs = ['black','blue']
        names = ['all', 'lockout']
        
        #
        lockouts = [False, True]
        ctr=0
        n_trials_both = []

        #
        data = np.load('/home/cat/concatenated_res.npz')
        conf_matrix_ten = data['conf_matrix_ten']
        n_trials_both.append(data['n_trials'])

        #
        sc=np.mean(conf_matrix_ten,axis=0) # mean of confusion matrix

        #
        confusion_10=sc.diagonal()
        confusion_s=conf_matrix_ten

        confusion_d=[]
        print (confusion_s.shape)
        for i in range(0,10):
            
            print (confusion_s[i].diagonal())
            confusion_d.append(confusion_s[i].diagonal())
        x_std=np.std(confusion_d,axis=0) #/(10**0.5)

        t = np.arange(-29.5,0.5,1)
        ax.plot(t,confusion_10, c=clrs[ctr])

        ax.fill_between(t, 
                         confusion_10+x_std,
                         confusion_10-x_std,
                         color=clrs[ctr],
                         alpha=.2,
                label=names[ctr])
        ctr+=1
        ax.set_ylim(0,1.0)
        #ax.set_xlim(-10,t[-1])

        ax.plot([t[0],t[-1]], [0.1,0.1], 'r--')

        
plot_decision_time()

(10, 30, 30)
[0.         0.         0.         0.02222222 0.         0.11111111
 0.04166667 0.         0.         0.09090909 0.04347826 0.03846154
 0.07142857 0.04166667 0.125      0.03030303 0.02222222 0.
 0.         0.         0.         0.05263158 0.         0.06818182
 0.02083333 0.         0.         0.03703704 0.         0.        ]
[0.         0.         0.03225806 0.05555556 0.0625     0.
 0.08       0.05263158 0.         0.         0.         0.
 0.1        0.07894737 0.05555556 0.03125    0.05714286 0.03846154
 0.02702703 0.         0.08571429 0.02272727 0.05263158 0.03448276
 0.06666667 0.         0.         0.07692308 0.         0.07142857]
[0.         0.         0.05882353 0.         0.08       0.
 0.03703704 0.03125    0.05882353 0.         0.04166667 0.2
 0.         0.2        0.         0.02439024 0.10526316 0.
 0.08333333 0.         0.07407407 0.02631579 0.         0.
 0.         0.0625     0.         0.         0.         0.        ]
[0.         0.05263158 0.         

In [4]:
data = np.load('/home/cat/concatenated_trials.npy')
data.shape

(218, 20, 900)

In [5]:
##############################
##### VISUALIZE A RESULT #####
##############################

import pandas as pd

#
fname = '/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb2_30Hz_whole_stack_pca30Components_window15sec_Xvalid10_Slidewindow30Frames_accuracy.pk'
accuracy = np.array(pd.read_pickle(fname))

# 
fig = plt.figure()
for k in range(len(accuracy)):
    ax=plt.subplot(2,5,k+1)
    acc = np.array(accuracy[k])
    print (acc.shape)

    if acc.shape[0]==0:
        continue
    t = np.arange(acc.shape[0])/30-14
    
    # 
    mean = acc.mean(1)
    plt.plot(t,mean,label=labels[k])
    std = np.std(acc)
    ax.fill_between(t, mean+std,
                   mean-std,
                  alpha=.2)

    # 
    plt.xlim(-14,15)
    plt.xlim(-10,10)
    plt.ylim(0.4,1.0)
    plt.plot([-14,15],[0.5,0.5],'--',c='black')
    
    plt.legend()
plt.show()

<ipython-input-5-d794d1f35346>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  accuracy = np.array(pd.read_pickle(fname))


(870, 10)
(870, 10)
(870, 10)
(870, 10)
(870, 10)
(870, 10)
(0,)
(870, 10)
(870, 10)
(870, 10)


In [12]:
# ######################################################
# ######## SET SHIFTS FOR LEVER PULL AND DLC DATA ######
# ######################################################

# # SET SHIFT FOR LEVER PULL
# # Find blue light on/off
# blue_light_threshold = 400  #Intensity threshold; when this value is reached - imaging light was turned on
# start_blue = 0; end_blue = len(aligned_images)

# if np.average(aligned_images[0])> blue_light_threshold:    #Case #1: imaging starts with light on; need to remove end chunk; though likely bad recording
#     for k in range(len(aligned_images)):
#         if np.average(aligned_images[k])< blue_light_threshold:
#             #self.aligned_images = self.aligned_images[k:]
#             end_blue = k
#             break
# else:                                                           #Case #2: start with light off; remove starting and end chunks;
#     #Find first light on
#     for k in range(len(aligned_images)):
#         if np.average(aligned_images[k])> blue_light_threshold:
#             start_blue = k
#             break

#     #Find light off - count backwards from end of imaging data
#     for k in range(len(aligned_images)-1,0,-1):
#         if np.average(aligned_images[k])> blue_light_threshold:
#             end_blue= k
#             break

# #
# filtered_filename = images_file[:-4]+'_'+selected_dff_filter+'_'+str(low_cut)+'hz_'+str(high_cut)+'hz.npy'
# if os.path.exists(filtered_filename):
#     try:
#         aligned_images = np.load(filtered_filename, allow_pickle=True)
#     except:
#         print ("aligned filtered images corrupt... recomputing: ", filtered_filename)
#         self.filter_data(root_dir, recording)
#         aligned_images = np.load(filtered_filename)
# else:
#     print ("aligned filtered images missing... recomputing: ", filtered_filename)
#     self.filter_data(root_dir, recording)
#     aligned_images = np.load(filtered_filename)

# aligned_images = aligned_images[start_blue:end_blue]


        
        

(71, 15, 901)


In [3]:
#############################################################
##################VISUALIZe ################
#############################################################
data=[]
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_10xvalid.npy').T)
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_50xvalid.npy').T)
#data.append(np.load('/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_100xvalid.npy').T)

fnames = [
    #'/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid10_Slidewindow30.npy',
    '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow30.npy',
    '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow90.npy',
    #     '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid20_Slidewindow30.npy'
    #'/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow10.npy'

]
# 
clrs=['black','blue','red']
ctr=0
fig=plt.figure()
for fname in fnames:
    data = np.load(fname).T
    
    sd = np.where(data>1)
    print ("sd: ", sd)
    
    idx = fname.find('Xvalid')
    idx2 = fname[idx:].find("_")
    xvalid = int(fname[idx+6:idx+idx2])
    print ("xvalid: ", xvalid)
    
    idx = fname.find('Slidewindow')
    idx2 = fname[idx:].find(".npy")
    swindow = int(fname[idx+11:idx+idx2])
    print ("swindow: ", swindow)
    
    
    
    #ax=plt.subplot(2,2,k+1)
    mean = data.mean(1)
    std = np.std(data,1)
    t=(np.arange(mean.shape[0]))/30-15.+swindow/30
    plt.plot(t,mean,c=clrs[ctr])
    plt.fill_between(t,mean+std, mean-std, color=clrs[ctr], alpha=.2,
                    label="xvalid: "+str(xvalid)+
                          '  swindow: '+str(swindow),
                    )
    
    plt.plot([t[0],t[-1]],[0.5,0.5],'--',c='black')
    plt.legend()
    plt.ylim(0.3,1.0)
    ctr+=1
plt.xlim(right=0)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IA1/SVM_Scores/IA1pm_Mar3_30Hz/SVM_Scores_IA1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95_Xvalid100_Slidewindow30.npy'

In [4]:
#####################################
##### PREDICT SVM DECISION TIME #####
#####################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10
svm.use_saved_model = False
svm.fname_saved_model = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb11_30Hz/analysis/00599_svm.pkl'

# 
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work

# 
lockouts = [False, True]
for lockout in lockouts:
    svm.lockout=lockout
    for id_ in names:
        svm.animal_id = id_
        svm.session_id = 'all'

        # 
        svm.predict()

# svm.process_data()

  0%|          | 0/71 [00:00<?, ?it/s]

 too few trials... skipping


  4%|▍         | 3/71 [00:01<00:27,  2.46it/s]

no such filename, skipping
 too few trials... skipping


 14%|█▍        | 10/71 [00:02<00:13,  4.64it/s]

...

 30%|███       | 33/110 [00:05<00:19,  3.93it/s]

no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping
 too few trials... skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
no such filename, skipping
 too few trials... skipping


 62%|██████▏   | 68/110 [00:14<00:21,  1.97it/s]

no such filename, skipping


100%|██████████| 110/110 [00:35<00:00,  3.07it/s]


In [18]:
################################################
################################################
################################################
fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()
print (svm.sessions)

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)

['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Jan29_30Hz'
 

Text(0.5, 0.98, 'AQ2')

In [12]:
import numpy as np
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb11_30Hz/IA1pm_Feb11_30Hz_0secNoMove_movements.npz',
              allow_pickle=True)

feature_init=data['feature_initiations']
print (feature_init.shape)

quiescent = data['feature_quiescent']
print (quiescent.shape, len(quiescent[2]))

labels = data['labels']
print (labels)

print (data['video_shift'])

(7, 20050)
(7,) 2960
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
[4.765 5.036 4.922]
